In [1]:
#PROJECT_NAME = 'ssd_mobilenet_v1_coco'
PROJECT_NAME = 'face'

CONFIG_FILE = 'ssd_mobilenet_v1_coco.config'
FINE_TUNE_CKPT = 'ssd_mobilenet_v1_coco_2017_11_17/model.ckpt'
NUM_CLASSES = 1


#CONFIG_FILE = 'ssd_inception_v2_coco.config'
#PROJECT_NAME = 'ssd_inception_v2_hand'
#FINE_TUNE_CKPT = 'ssd_inception_v2_coco_2017_11_17/model.ckpt'


%env PROJECT_NAME=$PROJECT_NAME


# Utility definitions
import subprocess
import os
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)
import re
import shutil
import os

bashPath = os.path.join(os.environ['PROGRAMFILES'], 'Git\\bin\\sh.exe')
if not os.path.exists(bashPath):
    raise Exception('Cannot find "{}".'.format(bashPath))

    
def runBash(cmd):
    subprocess.Popen([bashPath, 
                      '--login',
                      '-i', 
                      '-c', 
                      '{} ; read -n 1 -s -r -p "Press any key to exit..."'.format(cmd)],
                     creationflags=subprocess.CREATE_NEW_CONSOLE)
    
    
@register_cell_magic
def bash(line, cell=None):
    cmd = cell
    subprocess.Popen([bashPath, '--login','-i', '-c', '{} ; read -n 1 -s -r -p "Press any key to exit..."'.format(cmd)], 
                     creationflags=subprocess.CREATE_NEW_CONSOLE)
    
    
def replaceInFile(srcFile, destFile, replace):
    with open(srcFile, 'r') as f:
        text = f.read()
    
    for k, v in replace.items():
        patt = k.replace('.', '\\.')
        patt = re.compile(patt)
        match = patt.search(text)
        if match is None:
            raise Exception("Can't find '{0}' in config file.".format(k))

        text = re.sub(patt, v, text)
        print(match.group(0), "=>", v)

    with open(destFile, 'w') as f:
        f.write(text)

env: PROJECT_NAME=face


In [9]:
projectDir = 'projects' + '/' + PROJECT_NAME
if not os.path.exists(projectDir):  # Create project folder
    os.makedirs(projectDir)
    
replace = {
    'num_classes: \\d+': 'num_classes: {}'.format(NUM_CLASSES),
    'PATH_TO_BE_CONFIGURED/model.ckpt': "../../" + FINE_TUNE_CKPT,
    'PATH_TO_BE_CONFIGURED/mscoco_train.record': 'data/train.record',
    'PATH_TO_BE_CONFIGURED/mscoco_val.record': 'data/val.record',
    'PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt': 'data/label_map.pbtxt'
}

replaceInFile('models/research/object_detection/samples/configs/' + CONFIG_FILE,
              'projects/{}/{}.config'.format(PROJECT_NAME, PROJECT_NAME),
              replace)

num_classes: 90 => num_classes: 1
PATH_TO_BE_CONFIGURED/model.ckpt => ../../ssd_mobilenet_v1_coco_2017_11_17/model.ckpt
PATH_TO_BE_CONFIGURED/mscoco_train.record => data/train.record
PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt => data/label_map.pbtxt
PATH_TO_BE_CONFIGURED/mscoco_val.record => data/val.record


In [5]:
%%bash
# Protobuf Compilation

# From tensorflow/models/research/
cd models/research

protoc object_detection/protos/*.proto --python_out=.

In [22]:
%%bash
# Train
export PYTHONPATH=`pwd`/models/research/:`pwd`/models/research/slim:`pwd`/models/research/object_detection

# From project folder
cd projects/${PROJECT_NAME}

python ../../models/research/object_detection/train.py \
    --logtostderr \
    --pipeline_config_path=${PROJECT_NAME}.config \
    --train_dir=train

In [84]:
%%bash
# Export
export PYTHONPATH=`pwd`/models/research/:`pwd`/models/research/slim:`pwd`/models/research/object_detection

cd projects/${PROJECT_NAME}

rm -rf inference_graph

LAST_CKPT=`ls -A1r train/model* | head -n 1 | cut -d '.' -f 1-2`
echo Using: ${LAST_CKPT}

python ../../models/research/object_detection/export_inference_graph.py \
     --input_type image_tensor \
     --trained_checkpoint_prefix ${LAST_CKPT} \
     --pipeline_config_path ${PROJECT_NAME}.config \
     --output_directory inference_graph

In [2]:
%%bash
# Real-time detection using camera
export PYTHONPATH=`pwd`/models/research/:`pwd`/models/research/slim:`pwd`/models/research/object_detection

cd projects/${PROJECT_NAME}

python ../../realtime.py

In [4]:
%%bash
# Evaluation
export PYTHONPATH=`pwd`/models/research/:`pwd`/models/research/slim:`pwd`/models/research/object_detection

cd projects/${PROJECT_NAME}

CUDA_VISIBLE_DEVICES=-1 \
python ../../models/research/object_detection/eval.py \
    --logtostderr \
    --pipeline_config_path=${PROJECT_NAME}.config \
    --checkpoint_dir=train \
    --eval_dir=eval

In [24]:
%%bash
# Tensorboard
cd projects/${PROJECT_NAME}
tensorboard --logdir=.

# Cloud Training

In [98]:
# export YOUR_GCS_BUCKET=
YOUR_GCS_BUCKET = 'ross_bucket'
%env YOUR_GCS_BUCKET=ross_bucket

#SAMPLE_CONFIG_FILE = '../hands-detection/models/object_detection/samples/configs/faster_rcnn_resnet101_pets.config'
SAMPLE_CONFIG_FILE = '../hands-detection/models/object_detection/samples/configs/ssd_mobilenet_v1_pets.config'


replace = {
    'batch_size: \\d+': 'batch_size: 18',
    'num_classes: \\d+': f'num_classes: {NUM_CLASSES}',
    'PATH_TO_BE_CONFIGURED/model.ckpt': f'gs://{YOUR_GCS_BUCKET}/data/model.ckpt',
    'PATH_TO_BE_CONFIGURED/[a-zA-Z]+_train.record': f'gs://{YOUR_GCS_BUCKET}/data/train.record',
    'PATH_TO_BE_CONFIGURED/[a-zA-Z]+_val.record': f'gs://{YOUR_GCS_BUCKET}/data/val.record',
    'PATH_TO_BE_CONFIGURED/[a-zA-Z]+_label_map.pbtxt': f'gs://{YOUR_GCS_BUCKET}/data/label_map.pbtxt'
}

replaceInFile(SAMPLE_CONFIG_FILE,
              f'projects/{PROJECT_NAME}/{PROJECT_NAME}.config',
              replace)

env: YOUR_GCS_BUCKET=ross_bucket
batch_size: 24 => batch_size: 18
num_classes: 37 => num_classes: 1
PATH_TO_BE_CONFIGURED/model.ckpt => gs://ross_bucket/data/model.ckpt
PATH_TO_BE_CONFIGURED/pet_train.record => gs://ross_bucket/data/train.record
PATH_TO_BE_CONFIGURED/pet_val.record => gs://ross_bucket/data/val.record
PATH_TO_BE_CONFIGURED/pet_label_map.pbtxt => gs://ross_bucket/data/label_map.pbtxt


In [99]:
# Copy files to bucket

FINE_TUNE_CKPT = 'ssd_mobilenet_v1_coco_11_06_2017/model.ckpt'

runBash(f"""
gsutil.cmd cp projects/{PROJECT_NAME}/{PROJECT_NAME}.config gs://{YOUR_GCS_BUCKET}/data/{PROJECT_NAME}.config

for file in projects/{PROJECT_NAME}/data/*; do
    echo gsutil.cmd cp $file gs://{YOUR_GCS_BUCKET}/data/${{file##*/}}
done

for file in {FINE_TUNE_CKPT}.*; do
    echo gsutil.cmd cp $file gs://{YOUR_GCS_BUCKET}/data/${{file##*/}}
done
""")

In [92]:
%%bash 
# Package the Tensorflow Object Detection code

cd ../hands-detection/models/
export PYTHONPATH=`pwd`:`pwd`/slim:`pwd`/object_detection

protoc object_detection/protos/*.proto --python_out=.

python setup.py sdist
(cd slim && python setup.py sdist)

In [115]:
%%bash
#Training on google cloud ML
cd ../hands-detection/models/
export PYTHONPATH=`pwd`:`pwd`/slim:`pwd`/object_detection
export PATH='/c/Users/Ross/AppData/Local/Google/Cloud SDK/google-cloud-sdk/platform/bundledpython':$PATH

gcloud ml-engine jobs submit training `whoami`_object_detection_`date +%s` \
    --job-dir=gs://${YOUR_GCS_BUCKET}/train \
    --packages dist/object_detection-0.1.tar.gz,slim/dist/slim-0.1.tar.gz \
    --module-name object_detection.train \
    --region us-central1 \
    --config object_detection/samples/cloud/cloud.yml \
    -- \
    --train_dir=gs://${YOUR_GCS_BUCKET}/train \
    --pipeline_config_path=gs://${YOUR_GCS_BUCKET}/data/hand.config

In [16]:
%%bash
# Eval
cd ../hands-detection/models/
export PYTHONPATH=`pwd`:`pwd`/slim:`pwd`/object_detection
export PATH='/c/Users/Ross/AppData/Local/Google/Cloud SDK/google-cloud-sdk/platform/bundledpython':$PATH

gcloud ml-engine jobs submit training `whoami`_object_detection_eval_`date +%s` \
    --job-dir=gs://${YOUR_GCS_BUCKET}/train \
    --packages dist/object_detection-0.1.tar.gz,slim/dist/slim-0.1.tar.gz \
    --module-name object_detection.eval \
    --region us-central1 \
    --scale-tier BASIC_GPU \
    -- \
    --checkpoint_dir=gs://${YOUR_GCS_BUCKET}/train \
    --eval_dir=gs://${YOUR_GCS_BUCKET}/eval \
    --pipeline_config_path=gs://${YOUR_GCS_BUCKET}/data/hand.config

In [133]:
# Download last ckpt on google cloud ML
runBash(f"""
cd projects/{PROJECT_NAME}/

LAST_CKPT=`gsutil.cmd ls gs://{YOUR_GCS_BUCKET}/train/model.ckpt* | tail -n 1 | cut -d '.' -f 1-2`
gsutil.cmd cp ${{LAST_CKPT}}.* train/
read
""")

In [134]:
%%bash
# Export to a Tensorflow graph proto (old)
cd ../hands-detection/models/
export PYTHONPATH=`pwd`:`pwd`/slim:`pwd`/object_detection

cd ../../ObjectDetect/projects/${PROJECT_NAME}

rm -rf inference_graph
mkdir inference_graph

LAST_CKPT=`ls -A1r train/model* | head -n 1 | cut -d '.' -f 1-2`
echo Using ${LAST_CKPT} ...

python ../../../hands-detection/models/object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path ${PROJECT_NAME}.config \
    --checkpoint_path ${LAST_CKPT} \
    --inference_graph_path inference_graph/frozen_inference_graph.pb

In [138]:
%%bash
cd ../hands-detection/models/
export PYTHONPATH=`pwd`:`pwd`/slim:`pwd`/object_detection
cd ../../ObjectDetect/projects/${PROJECT_NAME}

python ../../realtime.py